In [1]:
#Import necessary libraries. Make sure to pip install missing ones.
import pandas as pd
import numpy as np
from numpy import nan
import pandas as pd
import re
from docx import Document
import docx
import time
from pymed import PubMed
from Bio import Entrez
import concurrent.futures
import random
from pubmed_lookup import PubMedLookup, Publication
import scispacy
import spacy
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import os
from docx.shared import Inches
from PIL import Image
import requests
import json
import urllib.request, urllib.error, urllib.parse
import json
import os
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib.colors import LinearSegmentedColormap

# Types of Ontologies- params : "ontologies" in find_pref_label function

In [ ]:
# import urllib.request, urllib.error, urllib.parse
# import json
# import os
# from pprint import pprint

# REST_URL = "http://data.bioontology.org"
# API_KEY = "d9885f69-dfed-44cd-81c3-f0591df67a03"


# def get_json(url):
#     opener = urllib.request.build_opener()
#     opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
#     return json.loads(opener.open(url).read())

# # Get the available resources
# resources = get_json(REST_URL + "/")

# # Get the ontologies from the `ontologies` link
# ontologies = get_json(resources["links"]["ontologies"])

# # Get the name and ontology id from the returned list
# ontology_output = []
# for ontology in ontologies:
#     ontology_output.append(f"{ontology['name']}\n{ontology['@id']}\n")

# # Print the first ontology in the list
# pprint(ontologies[0])

# # Print the names and ids
# print("\n\n")
# for ont in ontology_output:
#     print(ont)

# Required functions

In [7]:
# Adjust search terms to append to use pubmed advanced search.
# For guide on how to make your own ls, refer to query box at
# https://pubmed.ncbi.nlm.nih.gov/advanced/
def gene_to_search(element, fullName):
    ls = []
    ls.append("(" + element + "[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract])) \
    NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])")

    ls.append("(" + fullName + "[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract])) \
    NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])")
        
    return ls

def gene_fullName(gene_abbr):
    url = f'https://mygene.info/v3/query?q=symbol:{gene_abbr}&fields=name'
    time.sleep(0.1)
    response = requests.get(url)
    data = response.json()

    return data['hits'][0]['name']

def generate_wordcloud(input_text, nlp1, nlp2, nlp3):
    doc1 = nlp1(input_text)
    doc2 = nlp2(input_text)
    doc3 = nlp3(input_text)

    gene_pattern = r"^[A-Z]{1}[A-Za-z0-9_-]*[A-Za-z]{1}[A-Za-z0-9_-]*$"
    genes = [token.text for token in doc2 if re.match(gene_pattern, token.text) and token.pos_ == 'NOUN']
    
    entities1 = ['_'.join(ent.text.split()) for ent in doc1.ents if ent.label_ in 
                 {'ORGAN', 'CELL', 'DEVELOPING_ANATOMICAL_STRUCTURE', 'PATHOLOGICAL_FORMATION'}]
    entities2 = ['_'.join(ent.text.split()) for ent in doc2.ents if ent.label_ in 
                 {'DISEASE'}]
    entities3 = [ent.text for ent in doc2.ents if ent.label_ in 
                 {'TAXON'}]
    combined_entities = genes + entities1 + entities2 + entities3
    
    stem_cell_pattern = re.compile(r'\bstem cell\b', re.IGNORECASE)
    ipsc_pattern = re.compile(r'\bipsc\b', re.IGNORECASE)
    if re.search(stem_cell_pattern, input_text):
        combined_entities.append('stem_cell')
    if re.search(ipsc_pattern, input_text):
        combined_entities.append('iPSC')

    filtered_entities = []
    for entity in combined_entities:
        if not any([entity in other_entity and entity != other_entity for other_entity in combined_entities]):
            filtered_entities.append(entity)

    filtered_text = ' '.join(filtered_entities)

    if not filtered_text:
        img = Image.new('RGB', (400, 200), color='white')
        return img

    wordcloud = WordCloud(background_color='white', max_words=100, contour_width=3, contour_color='steelblue')
    wordcloud.generate(filtered_text)

    return wordcloud

def fetch_abstract(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml")
    records = Entrez.read(handle)
    try:
        abstract_sections = records["PubmedArticle"][0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"]
        abstract = "\n".join(str(section) for section in abstract_sections)
    except KeyError:
        abstract = "No abstract available"
    return abstract

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def search_query(query, fullName_list, gene, i):
    Entrez.email = "choyoungb@gmail.com"  
    time.sleep(0.1)
    handle = Entrez.esearch(db="pubmed", term=query, retmax=1000)
    time.sleep(0.1)
    record = Entrez.read(handle)
    pmid_list = record["IdList"]
    article_list = []

    for pmid in pmid_list:
        try:
            time.sleep(0.1)
            handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml")
            time.sleep(0.1)
            records = Entrez.read(handle)

            try:
                title = records["PubmedArticle"][0]["MedlineCitation"]["Article"]["ArticleTitle"]
                title = remove_html_tags(title)
                pub_date = records["PubmedArticle"][0]["MedlineCitation"]["Article"]["Journal"]["JournalIssue"]["PubDate"]
                article_ids = records["PubmedArticle"][0]["PubmedData"]["ArticleIdList"]
                doi_url = "NA"
                for article_id in article_ids:
                    if article_id.attributes["IdType"] == "doi":
                        doi_url = "https://doi.org/" + article_id
                if 'Year' in pub_date:
                        year = pub_date['Year']
                else:
                    year = None
                url = f"https://pubmed.ncbi.nlm.nih.gov/{pmid}"
                full_abstract = fetch_abstract(pmid)
                full_abstract = remove_html_tags(full_abstract)
                
                article_dict = {'info': "Url: " + url + "\n" + "DOI: " + doi_url + "\n\n" + "Title(" + year + "): "
                                + title + "\n\n" + full_abstract + "\n\n"}
                article_list.append(article_dict)             
                
            except IndexError:
                print(f"|An error occurred while fetching the article with PMID:{pmid}|")
                continue

        except Exception as e:
            print(f"|An error occurred while fetching the article with PMID:{pmid}|")
            print(e)

    search_df = pd.DataFrame(article_list)
    return search_df


# Load the library for wordcloud-keyword detection
* For more info, visit https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9028678/

In [3]:
#This step takes time
nlp1 = spacy.load("en_ner_bionlp13cg_md")
nlp2 = spacy.load("en_ner_bc5cdr_md")
nlp3 = spacy.load("en_ner_craft_md")

## PubMed Search Engine Description: https://www.nlm.nih.gov/vsearchfaq.html

In [4]:
# Replace email that you use for pubmed login
pubmed = PubMed(tool="MyTool", email="choyoungb@gmail.com")  # change to your email

# Replace tf with your list of gene names
original_file = pd.read_excel('TF.xlsx')
tf = original_file.iloc[:, 0].tolist()  # list of the first five genes
df = pd.DataFrame(columns=['gene', 'info'])
print(tf)
len(tf)

['ADNP', 'AHDC1', 'AR', 'ARHGEF2', 'ARID1B', 'ARID2', 'ARNT2', 'ARX', 'ASH1L', 'ATRX', 'BAZ2B', 'BCL11A', 'BRD4', 'BTAF1', 'BTRC', 'CAMK2A', 'CAMTA2', 'CARD11', 'CASZ1', 'CAT', 'CC2D1A', 'CDK8', 'CHAMP1', 'CHD1', 'CHD2', 'CHD3', 'CHD7', 'CHD9', 'CIC', 'CNOT3', 'CREBBP', 'CSDE1', 'CTCF', 'CTNNB1', 'CUL3', 'CUX1', 'CUX2', 'DDX3X', 'DEAF1', 'DEPDC5', 'DLX2', 'DLX3', 'DLX6', 'DNMT3A', 'DVL3', 'EBF3', 'EGR3', 'EN2', 'EP300', 'EP400', 'ERBIN', 'ERG', 'ESR2', 'ESRRB', 'FAN1', 'FBN1', 'FEZF2', 'FOXG1', 'FOXP1', 'FOXP2', 'GLIS1', 'GTF2I', 'HCFC1', 'HDAC4', 'HDAC8', 'HIVEP2', 'HIVEP3', 'HLA-DRB1', 'HMGN1', 'HNRNPD', 'HNRNPK', 'HNRNPU', 'HOXA1', 'IKZF1', 'JARID2', 'KAT2B', 'KAT6A', 'KAT6B', 'KDM2A', 'KDM2B', 'KDM3A', 'KDM5A', 'KDM5B', 'KDM5C', 'KLF16', 'KLF7', 'KMT2A', 'KMT2C', 'LMX1B', 'LZTR1', 'MACF1', 'MAPK3', 'MBD1', 'MBD3', 'MBD4', 'MBD6', 'MECP2', 'MED13', 'MED13L', 'MED23', 'MEF2C', 'MEIS2', 'MET', 'MINK1', 'MKX', 'MNT', 'MSL3', 'MSX2', 'MTF1', 'MYOCD', 'MYT1L', 'NCOA1', 'NCOR1', 'NFE2L3',

249

# GPT Filter X

In [8]:
df = pd.DataFrame(columns=['gene', 'info'])

for gene in tf:
    fullName = gene_fullName(gene)
    fullName_list = re.findall(r'[A-Za-z0-9]+', fullName)
    print(fullName_list)
    
    if len(fullName_list) > 1:
        fullName_string = ' '.join(fullName_list)
    else:
        fullName_string = fullName_list[0]
    
    query_list = gene_to_search(gene, fullName_string)
    i=0
    
    for search_phrase in query_list:
        print(i, search_phrase)
        
        search_df = search_query(search_phrase, fullName_list, gene, i)
        i += 1
        
        for index, row in search_df.iterrows():
            existing_row = df[df['info'] == row['info']]

            if not existing_row.empty:
                if gene not in existing_row['gene'].values[0]:
                    df.loc[existing_row.index, 'gene'] += f", {gene}"
            else:
                row['gene'] = gene
                df = pd.concat([df, row.to_frame().T], ignore_index=True)       
                
print("Dataframe complete")
                
doc = Document()
table = doc.add_table(rows=2 * len(df), cols=1)
row_idx = 0

for index, row in df.iterrows():
    table.cell(row_idx, 0).text = str(row["gene"])
    row_idx += 1
    abstract = row["info"].split("Title: ")[-1].split("\n\n", 1)[1]

    wordcloud = generate_wordcloud(abstract, nlp1, nlp2, nlp3)

    img_path = f"wordcloud_{index}.png"
    if isinstance(wordcloud, WordCloud):
        wordcloud.to_file(img_path)
    else: 
        wordcloud.save(img_path)


    table.cell(row_idx, 0).text = str(row["info"])
    paragraph = table.cell(row_idx, 0).paragraphs[0]
    run = paragraph.add_run()
    
    
    run.add_picture(img_path, width=Inches(6))
    os.remove(img_path)

    row_idx += 1


doc.save("output.docx")
print("WordFile Complete")



['activity', 'dependent', 'neuroprotector', 'homeobox']
0 (ADNP[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract]))     NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])
|An error occurred while fetching the article with PMID:27054228|
1 (activity dependent neuroprotector homeobox[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract]))     NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])
|An error occurred while fetching the article with PMID:27054228|
['AT', 'hook', 'DNA', 'binding', 'motif', 'containing', '1']
0 (AHDC1[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract]))     NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])
|An error occurred while fetching the article with PMID:34902026|
1 (AT hook DNA binding motif containing 1[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract]))     NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])
|An error occurred while

['catalase']
0 (CAT[Title/Abstract]) AND ((AUTISM[Title/Abstract]) OR (autistic[Title/Abstract]))     NOT (CANCER[Title/Abstract]) NOT (TUMOR[Title/Abstract])


KeyboardInterrupt: 

# Convert wordclouded docx to no-wordcloud

In [503]:
print('no-worldcloud doc generation')

# Read the existing 'output.docx' file
input_doc = Document('output.docx')

# Create a new docx file
output_doc = Document()

# Add a table with the same size as the input table
table = output_doc.add_table(rows=len(input_doc.tables[0].rows), cols=1)

# Iterate through the cells of the input table
for idx, cell in enumerate(input_doc.tables[0]._cells):

    # Copy the text from the input cell to the output cell
    table._cells[idx].text = cell.text

    # Remove images from the output cell
    for paragraph in table._cells[idx].paragraphs:
        for run in paragraph.runs:
            if run.element.rPr:
                for child in run.element.rPr.getchildren():
                    if child.tag.endswith('blip'):
                        child.getparent().getparent().getparent().getparent().remove(run.element.rPr)
                        
print('complete')

# Save the new docx file
output_doc.save('output_no_images.docx')


no-worldcloud doc generation
